# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [6]:
import math
from typing import Optional, Tuple, Dict

def solve_quadratic_choose_higher(B: float, V: float, A: float, F: float) -> Optional[float]:
    """
        Description:
        Helper funtion for avg_cost, performing the quadratic calculations. Full explanation in avg_cost.
    
        Solve B*Q^2 + (V - A)*Q + F = 0
        Q = [ -(V-A) += sqrt( (V-A)^2) - 4(B*F) ) ] / ( 2 * B )
        If two real roots, return the higher root, unless we don't have the inventory, then choose lower.
        If one real root (discriminant == 0), return that root.
        If no real roots, return None.
    """
    a = B
    b = V - A
    c = F
    if abs(a) < 1e-12:
        # Degenerate: linear equation b*Q + c = 0 -> Q = -c/b
        if abs(b) < 1e-12:
            return None
        return -c / b

    disc = b*b - 4*a*c
    if disc < 0:    #imaginary roots, disregard
        return None
    elif abs(disc) < 1e-12: #value under sqrt is zero -> one real root
        q = -b / (2*a)
        return q
    else:    #two real roots
        sqrt_disc = math.sqrt(disc)
        q1 = (-b + sqrt_disc) / (2*a)
        q2 = (-b - sqrt_disc) / (2*a)
        return max(q1, q2)


#Nondiagnostic version: only returns price and quantity
def avg_cost(A: float, B: float, V: float, F: float) -> Optional[int]:
    """  
        -Returns a suggested quantity of goods to produce and sell for this tick, assuming the demand graph is linear.
        -Returned quantity will set generated revenue equal to production cost (Net Profit = 0), as long as all units are sold
        -Returned quantity is rounded to the nearest whole number, so there is some margin of error that may result in 
        less than perfect results
        
        Equation for calculation:
        B*Q^2 + (V - A)*Q + F = 0
        Applying Quadratic Equation = 
        Q = [ -(V-A) +- sqrt( (V-A)^2) - 4(B*F) ) ] / ( 2 * B )
        If two real roots, return higher root to incentivise more sales.
        If one real root, return.
        If no real roots, apply linear_profit_max fallback to get production level with least amount of profit loss.
        Args:  
            A (float): Intercept of the Demand Graph (price at quantity zero)
            B (float): Slope of the Demand Graph 
            V (float): Variable cost per unit
            F (float): Fixed cost
            Q_Max: this is the maximum production quantity that the Industry is capable of producing
        Returns:  
            Q_Rounded (float): Calculated Quantity, rounded to nearest whole number
    """
    unrounded_q = 0.0
    q_root = solve_quadratic_choose_higher(B, V, A, F)
    #if result is infeasible, run adjusted linear profit fallback
    if q_root is not None: #real solution
        if q_root <= 0: # nonpositive = infeasible
            q_root = None
        else:
            # clip to demand-feasible max (price nonnegative)
            q_demand_max = A / B if B > 0 else q_root
            if q_root > q_demand_max:
                # root beyond demand support -> infeasible
                q_root = None
        unrounded_q = q_root
    else:
        #if there are no real roots, there are no profitable production levels
        #instead, minimize loss via linear profit max equation with Marginal Cost = V    
        q_adj = linear_profit_max(A, B, V, 0)

        if q_adj <= 0:
            # If fallback yields zero quantity, average cost is not defined (division by 0).
            q_adj = 0
        unrounded_q = q_adj
        
        
    if unrounded_q is not None:
        return round(unrounded_q)
    else:
        return None

#Nondiagnostic version: only returns price and quantity
def linear_profit_max(A, B, m, n) -> int:
    """  
        Description:
        -Returns a suggested quantity of goods to produce and sell for this tick, assuming the demand graph is linear.
        -Returned quantity will maximize profit for the industry, as long as all units are sold
        -Returned quantity is rounded to the nearest whole number, so there is some margin of error that may result in 
        less than perfect results
        
        Equation for calculation:
        Q = (A - m) / (2B + n)
        
        Args:  
            A (float): Intercept of the Demand Graph (price at quantity zero)
            B (float): Slope of the Demand Graph 
            m (float): Marginal Cost graph intercept
            n (float): Marginal Cost graph slope
            Note: if Variable cost scales linearly with quantity produced, 
                m = Variable Cost Per Unit
                n = 0
        Returns:  
            Q_Rounded (float): Calculated Quantity, rounded to nearest whole number
    """  
    denom = 2*B + n
    if denom == 0:
        raise ValueError("Denominator 2B + n is zero — examine boundaries.")
    Q_star = (A - m) / denom
    Q_feas = max(Q_star, 0.0)
    Q_Rounded = round(Q_feas)
    return Q_Rounded

def linear_price(A,B,Q) -> float:
    """  
        Returns a suggested price to sell goods, assuming the demand graph is linear and there is no competition
        
        Equation for Calculation:
        P = A - (B*Q)
        
        Args:  
            A (float): Intercept of the Demand Graph (price at quantity zero)
            B (float): Slope of the Demand Graph 
        Returns:  
            Price_Rounded (float): Calculated Price, rounded to two decimal places ($X.XX)
    """  
    P_at_Q = A - B * Q
    Price_Rounded = round(P_at_Q,2) #round to two decimal places
    return Price_Rounded

def main(args=None):
    upper_breakpoint = avg_cost(25.00, 0.001, 5.40, 4000)
    maximizing_quantity = linear_profit_max(25.00, 0.001, 5.40, 0)
    val_range = upper_breakpoint - maximizing_quantity
    lower_breakpoint = maximizing_quantity - val_range
    price = linear_price(25,0.001,maximizing_quantity)
    print(upper_breakpoint)
    print(maximizing_quantity)
    print(val_range)
    print(lower_breakpoint)
    print(price)
    
    return

main()

19394
9800
9594
206
15.2


In [1]:
import logging

logging.basicConfig(level=logging.INFO)
from engine.types.industry_type import IndustryType
from engine.types.demographic import Demographic


policies = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
    "subsidies": {itype.value: 0.0 for itype in IndustryType},
    "minimum_wage": 0.0,
}

demographics = {
    demo: {
        "income": {"mean": 300 + (i * 500), "sd": 10 + (i * 100)},
        "proportion": 1 / len(Demographic),
        "unemployment_rate": 0.1 - (i * 0.02),
        "spending_behavior": {
            itype.value: 1 / len(IndustryType) for itype in IndustryType
        },
        "current_money": {"mean": 500 + (i * 2000), "sd": 100},
    }
    for i, demo in enumerate(Demographic)
}

In [2]:
from engine.interface.controller import ModelController
controller = ModelController()
model_id = controller.create_model(num_people=100,demographics=demographics, starting_policies=policies,inflation_rate=0.001)

Incomes for lowerclass: [304.81963088 301.98240608 302.61496102 303.74095875 303.33275802
 299.75479647 298.28939663 309.23887583 304.07917532 291.80694979
 313.28921598 312.3017363  299.29845607 318.25307829 308.29501422
 297.50081467 284.38592748 287.95288403 311.8068449  295.46165041
 310.79026779 299.35481081 295.10117933 295.79989017 306.08726809
 296.19687853 311.92576013 288.7979997  299.91007133 301.5156026
 298.56198424 292.93437657 300.98166478]
Incomes for middleclass: [ 711.91239428  887.35449471  693.73322123  753.91443792  888.88955294
 1109.9755459  1008.05693509  929.77950765  726.29604813  707.8472514
  776.28623068  726.20575605  942.28653237  754.96212263  731.17908642
  801.59445781  703.56664627  739.69092865  815.47984336  918.17050111
  698.29841622  723.01266004  764.06194633  732.55920249  822.63957015
 1037.63817787  830.50237832  766.92952642  773.40751184  566.79689589
  537.53533076  924.66216434  529.59337377]
Incomes for upperclass: [1296.32061612 1450.91

In [3]:
controller.step_model(model_id,time=1)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Producing goods...NOT IMPLEMENTED


In [4]:
indicators = controller.get_indicators(model_id, start_time=1, end_time=0)
print(indicators)

   Week  Unemployment  GDP  IncomePerCapita  MedianIncome  HooverIndex  \
0     1           1.0    0       801.000001    801.000028            0   

   LorenzCurve  
0            0  
